# Download ECG data

This notebook downloads ECG data from the [MIT-BIH Arrhythmia Database Directory](https://archive.physionet.org/physiobank/database/html/mitdbdir/mitdbdir.htm)

Copyright 2020 Dr. Klaus G. Paul 

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [1]:
from IPython.display import display
from ipywidgets import IntProgress
import numpy as np
import os
import pandas as pd
import wfdb
import requests
import zipfile, io

Download the zip archive and extract all the files

In [159]:
r = requests.get("https://storage.googleapis.com/mitdb-1.0.0.physionet.org/mit-bih-arrhythmia-database-1.0.0.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
if not os.path.exists("./mit-data"):
    os.mkdir("./mit-data")
z.extractall("./mit-data")

This is not very generic, but the example does not need a concise dataset.

In [2]:
r = set()
for f in os.listdir("./mit-data/mit-bih-arrhythmia-database-1.0.0/"):
    s = f.split(".")[0][:3]
    if s.isdigit():
        r.add(s)

In [3]:
wIP = IntProgress(min=0,max=len(r))
display(wIP)
allAbnormalities = []
allData = []

for rec in r:
    record = wfdb.rdrecord('./mit-data/mit-bih-arrhythmia-database-1.0.0/{}'.format(rec))
    dfHB = pd.DataFrame(record.p_signal)
    dfHB.rename(columns={0:record.sig_name[0],1:record.sig_name[1]},inplace=True)
    dfHB["record"] = rec # this is the reference between time series and markup data
    dfHB["sample"] = dfHB.index
    # this is known
    freq = 360
    period = '{}N'.format(int(1e9 / freq))
    index = pd.date_range(pd.to_datetime("2020-01-01 12:00"), periods=len(dfHB), freq=period)
    dfHB["Timestamp"] = index
    # need to reduce the amount of data
    dfHB = dfHB[dfHB.Timestamp < pd.to_datetime("2020-01-01 12:02:30")]
    dfHB.index = dfHB["Timestamp"]
    # else bokeh may complain about identical names
    del dfHB.index.name
    dfHB.to_parquet("../data/{}.parquet".format(rec), use_deprecated_int96_timestamps=True)
    allData.append(dfHB)
    
    ann = wfdb.rdann('./mit-data/mit-bih-arrhythmia-database-1.0.0/{}'.format(rec),
                     extension='atr',return_label_elements=['symbol', 'label_store', 'description'])
    ann.create_label_map()
    dfAnn = pd.DataFrame({"annotation":ann.description,"sample":ann.sample,"symbol":ann.symbol})
    dfAnn = dfAnn[dfAnn["sample"] <= len(dfHB)]
    dfAnn = pd.merge(dfAnn,dfHB,on="sample")
    dfAnn["record"] = rec
    # else bokeh may complain about identical names
    del dfAnn.index.name
    # uncomment this if you think you need the individual files
    #dfAnn.to_csv("../data/ann.{}.csv".format(rec))
    #dfAnn[dfAnn.symbol != "N"].to_csv("../data/ann.abnormalities.{}.csv".format(rec))

    allAbnormalities.append(dfAnn[dfAnn.symbol != "N"][["Timestamp","annotation","symbol","record"]])
    
    wIP.value += 1
    
pd.DataFrame().append(allAbnormalities,sort=False).to_parquet("../data/abnormalities.parquet", use_deprecated_int96_timestamps=True)
#pd.DataFrame().append(allData,sort=False).to_parquet("../data/ecg.parquet", use_deprecated_int96_timestamps=True)

IntProgress(value=0, max=48)